# 🎯 Planning & Design Patterns with GitHub Models (Python)

## 📋 Learning Objectives

This notebook demonstrates advanced planning and design patterns for building intelligent agents using the Microsoft Agent Framework with GitHub Models. You'll learn how to create agents that can decompose complex problems, plan multi-step solutions, and coordinate sophisticated workflows.

**Advanced Capabilities You'll Master:**
- 🧠 **Strategic Planning**: Breaking complex tasks into manageable subtasks
- 🗺️ **Multi-Step Reasoning**: Sequential and parallel task execution patterns
- 🎯 **Goal-Oriented Design**: Agents that work toward specific objectives
- 🔄 **Adaptive Planning**: Dynamic strategy adjustment based on context

## 🎯 Planning Architecture Concepts

### Core Planning Components
- **Task Decomposition**: Breaking complex problems into smaller, manageable pieces
- **Execution Planning**: Determining optimal order and dependencies for subtasks
- **Resource Management**: Efficiently allocating tools and capabilities
- **Progress Monitoring**: Tracking completion and adapting to changes

### Design Patterns for Planning
- **Strategy Pattern**: Multiple planning approaches for different scenarios
- **Chain of Responsibility**: Sequential processing with fallback options
- **Command Pattern**: Encapsulated task execution with undo/redo capabilities
- **Observer Pattern**: Progress tracking and event-driven updates

## 🏗️ Technical Architecture

### Planning System Components
- **Microsoft Agent Framework**: Python implementation with advanced planning support
- **GitHub Models Integration**: High-performance reasoning and decision-making
- **Task Orchestration**: Coordinated execution of complex workflows  
- **State Management**: Persistent tracking of planning progress and results

### Planning Process Flow
```python
Complex Goal → Task Analysis → Subtask Decomposition → Execution Planning
                ↓                    ↓                    ↓
         Priority Assessment → Resource Allocation → Sequential Execution
                ↓                    ↓                    ↓
         Progress Monitoring → Adaptive Replanning → Goal Achievement
```

## 🧠 Planning Methodologies

### 1. **Hierarchical Task Planning**
- Top-down decomposition of complex objectives
- Nested goal structures with dependencies
- Recursive planning for sub-objectives
- Efficient resource allocation across levels

### 2. **Sequential Planning**
- Step-by-step task execution with clear dependencies
- Error handling and recovery at each stage
- Progress checkpoints and validation
- Rollback capabilities for failed steps

### 3. **Parallel Planning**
- Concurrent execution of independent tasks
- Resource synchronization and conflict resolution
- Performance optimization through parallelization
- Coordinated completion and result aggregation

## ⚙️ Prerequisites & Setup


**Required Dependencies:**
```bash

pip uninstall agent-framework -y
pip uninstall agent-framework-azure-ai -y

pip install -r ../../Installation/requirements.txt --constraint ../../Installation/constraints.txt -U
```

**Environment Configuration (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

## 🎨 Planning Design Patterns

### Goal-Oriented Planning
- **SMART Goals**: Specific, Measurable, Achievable, Relevant, Time-bound objectives
- **Milestone Tracking**: Progress measurement and validation checkpoints
- **Success Criteria**: Clear definitions of task completion
- **Adaptive Strategies**: Dynamic planning adjustment based on feedback

### Resource-Aware Planning
- **Capability Assessment**: Understanding available tools and their limitations
- **Load Balancing**: Optimal distribution of tasks across available resources
- **Constraint Management**: Working within system and API limitations
- **Performance Optimization**: Maximizing efficiency and minimizing latency

### Error-Resilient Planning
- **Failure Detection**: Early identification of planning or execution issues
- **Recovery Strategies**: Automatic fallback and alternative approaches
- **State Preservation**: Maintaining progress during error recovery
- **Graceful Degradation**: Partial success when full completion isn't possible

## 🚀 Advanced Planning Features

- **Meta-Planning**: Agents that plan how to plan
- **Collaborative Planning**: Multi-agent coordination for complex projects
- **Learning from Experience**: Improving planning strategies over time
- **Real-Time Adaptation**: Dynamic replanning based on changing conditions

## 📊 Use Cases & Applications

### Business Process Automation
- Project management and task scheduling
- Workflow optimization and resource allocation
- Strategic planning and decision support
- Process improvement and automation

### Research & Analysis
- Literature review and synthesis
- Data analysis pipeline planning
- Experimental design and execution
- Report generation and formatting

### Creative Projects
- Content creation workflows
- Multi-media project coordination
- Campaign planning and execution
- Event organization and management

Ready to build intelligent planning agents that can tackle complex, multi-step challenges? Let's architect some sophisticated problem-solving capabilities! 🧠✨

In [1]:
# 🧹 Clean Up Previous Installations (Optional)
# Uncomment these lines if you need to remove previous agent framework versions
# This ensures a clean installation of the latest version
# ! pip uninstall agent-framework -y
# ! pip uninstall agent-framework-azure -y
# ! pip uninstall agent-framework-foundry -y

In [2]:
# 📦 Install Required Packages (Optional)
# Uncomment this line to install all required dependencies
# The -U flag ensures packages are upgraded to latest versions
# ! pip install -r requirements.txt --constraint constraints.txt -U

In [3]:
# 🔌 Install Additional Dependencies (Optional)
# Uncomment to install WebSocket support and OpenTelemetry for monitoring
# websockets: For real-time communication
# opentelemetry-semantic-conventions-ai: For AI observability and monitoring
# ! pip install websockets opentelemetry-semantic-conventions-ai

In [4]:
# 📦 Import Standard Libraries
# os: For environment variable access
# dotenv: For loading environment variables from .env file
import os

from dotenv import load_dotenv

In [5]:
# 📝 Import Data Modeling Libraries
# Pydantic: For data validation and structured outputs
# BaseModel: Base class for creating data models
# Field: For adding metadata and validation to model fields
# List: Type hint for list collections
from pydantic import BaseModel, Field
from typing import List

In [6]:
# 🤖 Import Microsoft Agent Framework Components
# ChatMessage: For creating structured chat messages
# Role: Enum for message roles (user, assistant, system)
# ChatOptions: Configuration options for chat interactions
# OpenAIChatClient: Client for OpenAI-compatible APIs (GitHub Models)
from agent_framework import ChatMessage, Role, ChatOptions
from agent_framework.openai import OpenAIChatClient

In [7]:
# 🔧 Load Environment Variables
# Load configuration from .env file
# Required: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
load_dotenv()

True

In [8]:
# 🎯 Agent Configuration
# Define the agent's name and role

AGENT_NAME = "TravelAgent"

# 📋 System Instructions for Planning Agent
# This agent acts as a coordinator that decides which specialized agents to use
AGENT_INSTRUCTIONS = """You are a planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialized in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
"""


In [9]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [10]:
options = ChatOptions(response_format=TravelPlan)

In [11]:
client = OpenAIChatClient(base_url=os.environ.get("GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id=os.environ.get("GITHUB_MODEL_ID"))

In [12]:
agent =  client.create_agent(name= AGENT_NAME , instructions=AGENT_INSTRUCTIONS)

In [13]:
messages = [
        ChatMessage(role=Role.USER, text="Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne")
    ]

In [14]:
response = await agent.run(messages,response_format=TravelPlan)

In [15]:
response.messages[0].text

"To create a travel plan for a family of 4 from Singapore to Melbourne, I'll need to run the following agents:\n\n1. **FlightBooking**: To find and book flights for the family from Singapore to Melbourne.\n2. **HotelBooking**: To find and book suitable accommodations in Melbourne for the family.\n3. **CarRental**: To arrange for a car rental in Melbourne for convenient transportation.\n4. **ActivitiesBooking**: To suggest and book family-friendly activities in Melbourne.\n\nI'll proceed with running these agents."